# Building Deep Learning Applications with Keras: Recurrent Neural Networks


If we are talking about Deep Learning approaches for handling task data, one of the most common approaches is to use a Recurrent Neural Network (RNN). This networks act not only on their current input, but they have a memory composed on all the inputs it has already seen, as can be seen on the following image.

<table>
<tr>
<td>
<img align="middle" width='600' heith='100' src="images/lstm.png">
</td>
</tr>
</table>

These methods read text in a similar way as humans do. When we read one word in a phrase, we know the meaning of this word based on the words we have already read. Therefore, we are able to identify the context that the word is appearing. It is expected that a Recurrent Neural Network captures the same context information when handling text data.

We can also see in the above image that each word is converted into an **embedding** representation. An embedding is a vectorial representation of a word. It is used to map a word into a low dimensional space and to also capture semantic similarities between words. If you want to read more about word embeddings, please take a look at this [post](mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)

With all that said, let's implement our first Recurrent Neural Network in Keras. We will be using these network to deal with a sentiment analysis task.

## Sentiment Analysis with Recurrent Neural Networks in Keras

Sentiment analysis is an important task in the Natural Language Processing field. It can be used to understand how users are responding to some new technology, marketing campaign, political idea and any kind of application which users approval is highly correlated with success. Therefore, it is important to have models to accurately handle this kind of tasks.

Although when dealing with sentiment analysis, we can, for example, count the number of "positive" and "negative" words in a text and use that to judge the text polarity, there are some times that simple approaches like these one are not going to work. 

This happens a lot when we have a long text or text where the sentiment is captures not by single words appearing in it, but in the context that these words appear. One example, is movie reviews such as this one:

*I happened across "Bait" on cable one night just as it started and thought, "Eh, why not?" I'm glad I gave it a chance. "Bait" ain't perfect. It suffers from unnecessarily flashy direction and occasional dumbness. But overall, this movie worked. All the elements aligned just right, and they pulled off what otherwise could have been a pretty ugly film. Most of that, I think, is due to Jamie Foxx. I don't know who tagged Foxx for the lead, but whoever it was did this movie a big favor. Believable and amazingly likeable, Foxx glides through the movie, smooth as butter and funnier than          hell. You can tell he's working on instinct, and instinct doesn't fail him. The plot, while unimportant, actually ties together pretty well, and there's even a character arc through which Foxx's character grows as a person. Again, they could've slipped by without any of this, but it just makes things that much better. I'm surprised at the low rating for this. Maybe I just caught this move on the right night, or vice versa, but I'd give it a 7/10. Bravo, Mssr. Foxx.*

This a movie review with a positive polarity. We can see that this movie review has both positive and negative words, such as **dumbness**, **ugly**, **unimportant**, **hell**, **funnier**, and other words. Also we can see that the text has combination of words that could not be analyzied alone, for example **pretty ugly**, which is an oxymoron. **Pretty** and **ugly** have opposite meanings, but in that sentence, we see that **pretty** is being used to but emphasis on **ugly**. Another similar expression here is **ain't perfect**, where **ain't** act as a negation to the word **perfect** that comes right after this word.

Therefore, we can see that to analyze such a text, we need more than counting ocurrences of words. We need to understand the context that these words appear in. That's why we will be using a Recurrent Neural Network for this problem.

With that said, the task we will solve is to analyse sentiment in movie reviews similar to the one we have analyzed above. We will use th **IMDB dataset**, which contains 25000 reviews for training and 25000 reviews for testing the model. This dataset is also perfectly balances, ,meaning that we have exactly 125000 reviews with positive sentiment and 12500 reviews with negative sentiment in the training data. The dataset is also perfectly balanced on the test set.

<table>
<tr>
<td>
<img align="middle" width='600' heith='100' src="images/imdb.png">
</td>
</tr>
</table>

First, let's load this dataset in memory and take a closer look at it.

In [ ]:
from keras.datasets import imdb

"""
The size of our vocabulary or the number of unique words that we will use to solve our problem.
"""
num_words = 20000


(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)
print('Number of train movie reviews: {}'.format(len(x_train)))
print('Number of test movie reviews: {}'.format(len(x_test)))

Now let's take a look at an example of movie review in our training set.

In [ ]:
print('Example of a movie review:\n{}\n'.format(x_train[0]))

Wait a minute, we are going to classify movie reviews, so where are the words ?? And are those numbers ??

Remember our **num_words** variable ? This variable dictates how many words are going to be used to solve our problem. What this means is that we have a list of valid words that we can use, for example, our list could be something like:

valid_words = [good, bad, the, is, great, terrible]

We can see that each of these words have a position in that list, for example, the word good has position 1, the word terrible has position 6. These positions are the number we see in the above example. Every word in our movie review has been mapped to the position it has in the valid words list.

But now you may be wondering, why would we replace the words by their position in the list ? Well, remember from the Recurrent Neural Network image in the beggining of our document that we must convert our word into **embedding**. An embedding is a vector representation of our word, for example, we can convert the word **good** into:

good = [0.3, 0.2, 0.6, -0.61, 012]

Every word in our valid_words list has an associated vector for it. Therefore, we have an embedding matrix, with dimensions V X D, where V is the size of our valid words list and D is the dimension of our embedding vector. This dimension represents the amount of infomation we are keeping for a word. With more dimensions, more information we can store about a word, for example, that it is similar to the words **great** and **terrific** or that it has positive sentiment. However, a bigger dimension also means that he have a model that is more slow to train and a model with more flexibility, meaning that can overfit more easily to a task.

With all that said, the numbers is the list are the positions of the words in valid_words list and these numbers are used to select a row in the embedding matrix, allowing us to turn a word into a **embedding** representation.

Now, before we train our model, we need to perform one last step. To understand what this step is, take a look at the size of three consecutive reviews in our training data:

In [ ]:
print([len(review) for review in x_train[:3]], sep=' ')

We can see that each movie review has a different size. This makes a lot of sense, since different reviews should not have the same number of words. However, to train our model with more than one movie review at the same time, we cannot allow movie reviews to have different sizes. To solve this problem, we will pad our movie revies.

But what do I mean by padding ? Imagine that we have the following movie reviews:

98, 52, 58, 12, 15

86, 52, 10

90, 72, 36, 78, 92, 42

Remember that each of these numbers is the word position in the valid_words list. Now, when we pad our sentences, this is what happens:

00, 98, 52, 58, 12, 15

00, 00, 00, 86, 52, 10

90, 72, 36, 78, 92, 42

We can see that we have added the **00** number before each movie review, allowing the reviews to have the same size. This **00** token is the **pad token** and it carries no meaning. Therefore, our model will ignore these tokens when we see than.

Now to pad our reviews, we can do the following step in keras:

In [ ]:
from keras.preprocessing import sequence

"""
maxlen refers to the maximum size of a movie review in our dataset. This means that reviews with more
than 80 words in it will be cutted to 100 words. Also, reviews with fewer words than 100 will be padded
with our pad tokens
"""
maxlen = 100

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

Now, let's see the size of our three consecutive movie reviews again:

In [ ]:
print([len(review) for review in x_train[:3]], sep=' ')

We can see now that the reviews have the same size.

Finaly, we can implement our model. Our model will consist of three keras layers, an Embedding layer, to map each word
into an embedding array. A [LSTM](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) layer, which is a Recurrent Neural Network that better handle long dependecies between words, and a Dense layer with the sigmoid activation, to turn the output of our Recurrent Neural Network into a binary probability of a review being positive or negative.

First, let's define a class to hold the model hyperparameters:

In [ ]:
class RecurrentConfig:

    def __init__(self,
                 batch_size=32,
                 embedding_size=50,
                 num_words=20000,
                 lstm_units=64,
                 num_classes=1,
                 epochs=5):

        """
        Holds Recurrent Neural Network model hyperparams.

        :param batch_size: batch size for training
        :type batch_size: int
        
        :param embedding_size: The dimension of our embedding.
                               Recall that the embedding is a V X D matrix, where V is the number of
                               words in the vocabulary and D is the dimension of the embeddings.
                               This variable represents the D value in the embedding matrix.
        :type embedding_size:  int
        
        :param num_words: The size of our vocabulary or the number of words in the word_list variable.
        :type num_words:  int
        
        :param lstm_units: The number of units in the LSTM layer.
        :type num_words:  int
    
        :param num_classes: number of classes in the problem
        :type epochs: int
        
        :param epochs: number of epochs
        :type epochs: int
        """
        self.batch_size = batch_size
        self.embedding_size = embedding_size
        self.num_words = num_words
        self.lstm_units=lstm_units
        self.num_classes = num_classes
        self.epochs = epochs

    def __str__(self):
        status = ''
        status += 'batch size: {}\n'.format(self.batch_size)
        status += 'embedding size: {}\n'.format(self.embedding_size)
        status += 'num words: {}\n'.format(self.num_words)
        status += 'lstm units: {}\n'.format(self.lstm_units)
        status += 'num classes: {}\n'.format(self.num_classes)
        status += 'epochs: {}\n'.format(self.epochs)

        return status


In [ ]:
model_config = RecurrentConfig()
print(model_config)

Now, with the config class defined, we can create our model by writting our code in the **build_graph** method bellow. Here we will define our model and build it.

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense

from model.model import Model


class RecurrentModel(Model):
    
    def build_model(self):
        self.model = Sequential()
        
        ### YOUR CODE HERE

        ### END YOUR CODE
        
        self.model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        
        print(self.model.summary())

Now, let's see our model summary:

In [ ]:
model = RecurrentModel(model_config)
model.build_model()

You are supposed to see three layers in the model, an Embedding, a LSTM and a Dense layer. If your model has all of this layers, it is time to see it in action:

In [ ]:
train_data = (x_train, y_train)
model_history = model.fit(train_data)

Now let's evaluate our model in the test set and understand if our model is generalizing well.

In [ ]:
test_data = (x_test, y_test)
loss, accuracy = model.evaluate(test_data)

print('Test loss: {:.2f}'.format(loss))
print('Test accuracy: {:.2f}'.format(accuracy))

It is expected that our model has a similar accuracy on the test set than the one achieved on the train dataset. It is expected that we should achieve around 89% accuracy on the training data and 80% on the test set.

Although the test accuracy is a good indicator of how our model is performing, there is an important plot we must also observe to undertand how our model is performing, it is named "the train vs validation" graph. In this graph, we plot for each epoch the train loss and the validation loss.

For our model, we can see this graph below:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def plot_train_validation_graph(train_values, validation_values, title, ylabel, legend_loc='lower right'):
    plt.plot(train_values)
    plt.plot(validation_values)
    plt.title(title)
    plt.ylabel(ylabel)
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc=legend_loc)
    
plot_train_validation_graph(
    train_values=model_history.history['loss'],
    validation_values=model_history.history['val_loss'],
    title='Train loss vs Validation loss',
    ylabel='loss',
    legend_loc='upper right'
)

The **model_history** object holds information of variables collected during the training step of our model. This means that we can also plot the accuracy comparisons between the train and validation sets:

In [ ]:
plot_train_validation_graph(
    train_values=model_history.history['acc'],
    validation_values=model_history.history['val_acc'],
    title='Train Accuracy vs Validation Accuracy',
    ylabel='accuracy'
)

We can see that our model loss and train accuracy start to diverge from each other. This graph can shows us some troubles in our model. For example, imagine that you plot your graph and it looks like this:

In [ ]:
train_accuracy = [0.3, 0.4, 0.4, 0.45, 0.48]
val_accuracy = [0.2, 0.3, 0.3, 0.4, 0.4]

plot_train_validation_graph(
    train_values=train_accuracy,
    validation_values=val_accuracy,
    title='Train Accuracy vs Validation Accuracy',
    ylabel='accuracy'
)

In this graph, we can see that neither our train or validation set achieves high accuracy on the sets. This can be an indicator that our model is suffering from **underfitting**. This means that our model is not robust enough to handle our current data. 

Now, imagine that our graph is showing this type of behavior:

In [ ]:
train_accuracy = [0.3, 0.6, 0.8, 0.95, 1.0]
val_accuracy = [0.2, 0.3, 0.3, 0.4, 0.4]

plot_train_validation_graph(
    train_values=train_accuracy,
    validation_values=val_accuracy,
    title='Train Accuracy vs Validation Accuracy',
    ylabel='accuracy'
)

Now, we can see that our model is achieving a perfect accuracy in the training set, meaning that it can correctly predict the entire training set. However, this is not happening for our validation set, where the accuracy is quite low. In that case, our model is suffering of **overfitting**, meaning that the model is too robust for our dataset. In that case the model is capturing noise in our data, but it's not generalizing well to unseen data, which is a huge problem.

For both **underfitting** and **overfitting** there are some strategies we can use to solve this type of problem:
    
**Underfitting**
 
* Use a more complex model: Maybe a more robust model will be better on the task. Try adding more layers or increasing the  existent number of units in the model's layers.
* Add more variables to the data: Maybe the features we are using are not enough to fit the data well. Therefore we should add more features to the dataset.

**Overfitting**

* Try using a simpler model: Maybe your model is too robust for the task. Try removing some layers from your model or reducing the number of units in the model's layers.
* Add regularization strategies: Add regularization techniques to your model such as [dropout](https://keras.io/layers/core/#dropout) or L2 regularization on the model weights. To learn more about regularizers, take a look at this [post](https://www.analyticsvidhya.com/blog/2018/04/fundamentals-deep-learning-regularization-techniques/)
* Collect more data: If even reducing the model and adding regularizers to your model doesn't work, maybe the best approach is too collect more data. Actually, this is almost always the approach which will yield the best results, but it can also be costly. If that is the case, try the other two approaches before this one.

With all that said, let's now move to a more advanced problem using Recurrent Neural Networks on the part two of this notebook :)